# Segment a Wav Dataset into Frames of n Seconds with Overlapping

## GTzan Dataset

http://marsyasweb.appspot.com/download/data_sets/

The dataset consists of 1,000 audio tracks each 30 seconds long.

It contains 10 genres, each represented by 100 tracks.

The tracks are all 22,050Hz Mono 16-bit audio files in .wav format.


##### 8 May 2019
#### Alessandro L. Koerich

In [1]:
import numpy as np
import os
import soundfile as sf
import math

import matplotlib.pyplot as plt
import pylab as py
from scipy import fftpack
from scipy import signal
from pylab import *
import scipy.io.wavfile

In [2]:
# Path to source WAV files already split into folds
path2='./genres_3s'

# Path to destination (segmentated WAV files) 
# Win_length_overlapping
path3='GTzan_3f_Win_110250_75_safaa'

## Build a list of WAV files

In [3]:
# Build a list of WAV files
file_list = []
i = 0 
for path, subdirs, files in os.walk( path2 ):
    for name in files:
        file_list.append( os.path.join(path, name) )
        i += 1
print("Files processed: "+str(i) )

Files processed: 1000


In [4]:
file_list

['./genres_3s/fold2/metal/metal.00069.wav',
 './genres_3s/fold2/metal/metal.00073.wav',
 './genres_3s/fold2/metal/metal.00094.wav',
 './genres_3s/fold2/metal/metal.00076.wav',
 './genres_3s/fold2/metal/metal.00051.wav',
 './genres_3s/fold2/metal/metal.00033.wav',
 './genres_3s/fold2/metal/metal.00080.wav',
 './genres_3s/fold2/metal/metal.00083.wav',
 './genres_3s/fold2/metal/metal.00038.wav',
 './genres_3s/fold2/metal/metal.00032.wav',
 './genres_3s/fold2/metal/metal.00009.wav',
 './genres_3s/fold2/metal/metal.00019.wav',
 './genres_3s/fold2/metal/metal.00016.wav',
 './genres_3s/fold2/metal/metal.00063.wav',
 './genres_3s/fold2/metal/metal.00054.wav',
 './genres_3s/fold2/metal/metal.00095.wav',
 './genres_3s/fold2/metal/metal.00081.wav',
 './genres_3s/fold2/metal/metal.00056.wav',
 './genres_3s/fold2/metal/metal.00008.wav',
 './genres_3s/fold2/metal/metal.00050.wav',
 './genres_3s/fold2/metal/metal.00091.wav',
 './genres_3s/fold2/metal/metal.00070.wav',
 './genres_3s/fold2/metal/metal.

## Check if the audio samples have at least 5 seconds

In [5]:
# This block of code is to make sure that all of the audio samples have at least XX seconds

# For a sample rate = 22,050 
#  1s = 22,050 x 1  =  22,050 samples 
#  5s = 22,050 x 5  = 110,250 samples
# 10s = 22,050 x 10 = 220,500 samples

length = list()
i      = 0
avg    = 0

for file in file_list:
    data, samplerate = sf.read( file )
    #print(str(samplerate))
    if len(data) <= 110250:                         
            print("Audio with less than 5s: "+str(file) )
    #computer average lenght of files
    avg = avg + len(data)
    length.append(len(data))
    i += 1

print( "Files processed: "+str(i) )

print( "Average file length: "+str(avg/i)+" samples   "+str(avg/i/samplerate)+"s   "+str(avg/i/samplerate/60)+"min" )

print( "Max length: "+str(max(length))+" samples   "+str(max(length)/samplerate)+"s   "+str(max(length)/samplerate/60)+"min" )

print( "Min length: "+str(min(length))+" samples   "+str(min(length)/samplerate)+"s   "+str(min(length)/samplerate/60)+"min" )

Files processed: 1000
Average file length: 662030.846 samples   30.024074648526078s   0.5004012441421013min
Max length: 675808 samples   30.648888888888887s   0.5108148148148148min
Min length: 660000 samples   29.931972789115648s   0.4988662131519275min


## Sliding Window Setup

In [6]:
# window  = size of window (1,600 - 8,000 - 16,000 - 32,000, etc.)
# overlap = percentage of overlapping (0, 0.25, 0.50, 0.75, etc.)

# sample rate = 22,050
#  1s = 22,050 x 1 =  22,050 samples
#  5s = 22,050 x 5 = 110,250
# 10s = 22,050 x 10 = 220,500

def window(data, window, overlap):
    
    overlap = 1 - overlap
    print ( str( window * overlap) )
    print ( str( len(data) / (window*overlap) ) )

    # end = int( ceil( ( len(data) / (window*overlap) ) ) )
    end = int( ceil( ( (len(data) - window ) / (window*overlap) ) ) )
    print(" End: "+str(end) )
    
    # Make a 2-D array of samples
    windowed = []
    if end == 0:
        end = end + 1
    for i in range(0, end):
        
            if len( data[i*( int( window*overlap ) ):i*( int( window*overlap) ) + window] ) == window:
                windowed.append(data[i*( int( window*overlap ) ):i * ( int( window*overlap ) ) + window ] )
                # print("Done")
                
            if len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])<window:
                spch = np.zeros(window)
                spch[0:len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])]=data[0:len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])]
                spch[len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])+1:window]=data[len(data[i*(int(window*overlap)):i*(int(window*overlap))+window])-1]
                windowed.append(spch)
                print("Warning: shorter than frame size-> zero padding")
                
    return (windowed)

## Energy of the Audio Signal

In [7]:
# Compute the signal energy
def energy(samples):
    return np.sum(np.power(samples, 2.)) / float(len(samples))

## Write Metadata in a CSV File

In [8]:
# WRITE TO FILE METADATA

import csv

filename="GTzan3f_Spltr_110_2Voc_safaa.csv"

fl = open(filename, 'w')

writer = csv.writer(fl)

values = ['track_id','frame_id','fold','genre']

writer.writerow(values)

30

## Segment Audio Files with the Sliding Window

### Each segment is saved on disk with the appropriate prefix indicating the segment number (_segment number).

### Segment of low energy (possibly silence) are discarded

In [12]:
# Audio files are segmented and each segment is saved on disk with the approporiate prefix indicating the segment
# number.
# np.boxcar(frame_size)
# sample rate = 22,050
#  1s = 22,050 x 1 =  22,050
#  5s = 22,050 x 5 = 110,250
# 10s = 22050 x 10 = 220,500

# Split 30s audio samples into 5s segments with an overlapping of 75%
# 21 segments per audio sample
frame_size  = 110250
frame_over  = 0.75
track_id    = 1

for file in file_list:
    data , samplerate = sf.read( file )
    print ("--------------")
    print ("Sample Rate: "+str(samplerate)+"   Length: "+str(data.shape))
    print (str( file ) )
        
    frame_id = 1
            
    windowed = window( data, frame_size, frame_over )

    for i in range(len( windowed ) ):
        # print ("Size: "+str(windowed[i].shape) )
        # print ("-----------------------")
        spch = windowed[i] * signal.boxcar(frame_size)
        
        
        ##### Evaluate Energy
        ##### Only save a segment if it has enough energy
        if energy(spch) > 1e-7:
        
            seg_name_new = str(track_id)+'_'+str(frame_id)
            # seg_name_new = str(track_id)+'_'+str(artist_id)+'_'+str(frame_id)
            # seg_name     = str(i)+'_'+file[9:].split('/')[4]
            directory = path3+'/'+file[12:].split('/')[0]+'/'+file[13:].split('/')[1]
            
            # Verify if it exists, otherwise create it
            if not os.path.exists(directory):
                os.makedirs(directory)
            
        
            sf.write( path3+'/'+file[12:].split('/')[0]+'/'+file[13:].split('/')[1]+'/'+seg_name_new+".wav", spch, samplerate ) 
            # sf.write( path3+'/'+file[9:].split('/')[3]+'/'+seg_name, spch, samplerate ) 
            print( path3+'/'+file[12:].split('/')[0]+'/'+file[13:].split('/')[1]+'/'+seg_name_new+".wav")
        
            fold_id = file[12:].split('/')[0]
        
            genre_id = str(file[12:].split('/')[1])
        
            # Write a line in .CSV file
            values = [str(track_id), str(frame_id), str(fold_id), str(genre_id) ]  
        
            frame_id += 1
        
            writer.writerow(values)
            
        else:
            print("Energy = "+str(energy(spch)*100)+"%   - Silence file: "+str(i) )
        
    track_id += 1
        

--------------
Sample Rate: 22050   Length: (661504,)
./genres_3s/fold2/metal/metal.00069.wav
27562.5
24.000145124716553
 End: 21
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_1.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_2.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_3.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_4.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_5.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_6.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_7.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_8.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_9.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_10.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_11.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_12.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_13.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_14.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_15.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_16.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/1_17.wav
GTzan_3f_Win_110250_75_safaa/

GTzan_3f_Win_110250_75_safaa/fold2/metal/8_1.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_2.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_3.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_4.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_5.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_6.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_7.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_8.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_9.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_10.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_11.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_12.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_13.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_14.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_15.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_16.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_17.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_18.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_19.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/8_20.wav
GTzan_3f_

GTzan_3f_Win_110250_75_safaa/fold2/metal/15_6.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_7.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_8.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_9.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_10.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_11.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_12.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_13.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_14.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_15.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_16.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_17.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_18.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_19.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_20.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/15_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
./genres_3s/fold2/metal/metal.00095.wav
27562.5
24.010666666666665
 End: 21
GTzan_3f_Win_110250_75_safaa/fold2/metal/16_1.wav
GTzan_3f

GTzan_3f_Win_110250_75_safaa/fold2/metal/22_5.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_6.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_7.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_8.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_9.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_10.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_11.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_12.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_13.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_14.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_15.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_16.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_17.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_18.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_19.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_20.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/22_21.wav
--------------
Sample Rate: 22050   Length: (661504,)
./genres_3s/fold2/metal/metal.00071.wav
27562.5
24.000145124716553
 End: 21
GTzan_3f

GTzan_3f_Win_110250_75_safaa/fold2/metal/29_3.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_4.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_5.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_6.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_7.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_8.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_9.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_10.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_11.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_12.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_13.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_14.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_15.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_16.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_17.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_18.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_19.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_20.wav
GTzan_3f_Win_110250_75_safaa/fold2/metal/29_21.wav
--------------
Sample Rate: 22050   Le

...

GTzan_3f_Win_110250_75_safaa/fold1/rock/969_11.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_12.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_13.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_14.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_15.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_16.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_17.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_18.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_19.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_20.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/969_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
./genres_3s/fold1/rock/rock.00068.wav
27562.5
24.010666666666665
 End: 21
GTzan_3f_Win_110250_75_safaa/fold1/rock/970_1.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/970_2.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/970_3.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/970_4.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/970_5.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/970_6.wav
GTzan_3f_Wi

GTzan_3f_Win_110250_75_safaa/fold1/rock/976_14.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/976_15.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/976_16.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/976_17.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/976_18.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/976_19.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/976_20.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/976_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
./genres_3s/fold1/rock/rock.00003.wav
27562.5
24.010666666666665
 End: 21
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_1.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_2.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_3.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_4.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_5.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_6.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_7.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_8.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/977_9.wav
GTzan_3f_Win_1

GTzan_3f_Win_110250_75_safaa/fold1/rock/983_12.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_13.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_14.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_15.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_16.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_17.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_18.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_19.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_20.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/983_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
./genres_3s/fold1/rock/rock.00015.wav
27562.5
24.010666666666665
 End: 21
GTzan_3f_Win_110250_75_safaa/fold1/rock/984_1.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/984_2.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/984_3.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/984_4.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/984_5.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/984_6.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/984_7.wav
GTzan_3f_Win

GTzan_3f_Win_110250_75_safaa/fold1/rock/990_11.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_12.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_13.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_14.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_15.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_16.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_17.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_18.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_19.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_20.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/990_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
./genres_3s/fold1/rock/rock.00066.wav
27562.5
24.010666666666665
 End: 21
GTzan_3f_Win_110250_75_safaa/fold1/rock/991_1.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/991_2.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/991_3.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/991_4.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/991_5.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/991_6.wav
GTzan_3f_Wi

GTzan_3f_Win_110250_75_safaa/fold1/rock/997_11.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_12.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_13.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_14.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_15.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_16.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_17.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_18.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_19.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_20.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/997_21.wav
--------------
Sample Rate: 22050   Length: (661794,)
./genres_3s/fold1/rock/rock.00030.wav
27562.5
24.010666666666665
 End: 21
GTzan_3f_Win_110250_75_safaa/fold1/rock/998_1.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/998_2.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/998_3.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/998_4.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/998_5.wav
GTzan_3f_Win_110250_75_safaa/fold1/rock/998_6.wav
GTzan_3f_Wi

In [13]:
print("Audio segmentation finished")

Audio segmentation finished
